In [1]:
using DifferentialEquations
using LinearAlgebra
using Plots
using DataInterpolations
using Symbolics
using Symbolics: derivative
using WindowFunctions

In [2]:
function mprod(x)
    Re = I(2)  
    Im = [0 -1;
           1 0]   
    M = kron(Re,real(x)) + kron(Im,imag(x));
    return M   
end

function inprod(x)
    a = x[1:n]
    b = x[(n+1):(2*n)]
    P = [a*a'+b*b' -(a*b'+b*a');
         a*b'+b*a' a*a'+b*b']
    return P
end

inprod (generic function with 1 method)

In [3]:
n = 2
m = 1

H0 = [1 0;
     0 -1]

H1 = [0 -1im;
      1im 0]


nu = eigvecs(H0)
nu1 = nu[:,1]
nu2 = nu[:,2]

x0 = [nu1;0*nu1]
xf = [nu2;0*nu2]

Qe = I(2*n) - inprod(xf)
Qm = 0.0

# Pf = 1.6322*(I(2*n) - inprod(xf))
Pf = I(2*n) - inprod(xf)

4×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0

In [4]:
@variables xx[1:(2*n)] uu[1:m] qq[1:(2*n)]

f(xx,uu) = collect(mprod(-1im*(H0 + uu[1]*H1))*xx)

l(xx,uu) = 0.01/2*(uu[1]^2) + Qm*1/2*(xx[2]^2 + xx[4]^2)

# M(xx) = 1/2*xx'*Pf*xx
λ
L(xx,uu,qq) = collect(l(xx,uu) + transpose(qq)*f(xx,uu))

function jacobian(dx, f, args...; inplace = false)
    f_sym = f(args...)
    fx_sym = cat(map(1:length(dx)) do i
        map(1:length(f_sym)) do j
            derivative(f_sym[j], dx[i])
        end
    end...; dims = ndims(f_sym)+1)

    return eval(build_function(fx_sym, args...)[inplace ? 2 : 1])
end

fx = jacobian(xx, f, xx, uu)
fu = jacobian(uu, f, xx, uu)


lx = jacobian(xx, l, xx, uu)
lu = jacobian(uu, l, xx, uu)

lxx = jacobian(xx, lx, xx, uu)
lxu = jacobian(uu, lx, xx, uu)
luu = jacobian(uu, lu, xx, uu)

Lx = jacobian(xx, L, xx, uu, qq)
Lu = jacobian(uu, L, xx, uu, qq)

Lxx = jacobian(xx, Lx, xx, uu, qq)
Lxu = jacobian(uu, Lx, xx, uu, qq)
Luu = jacobian(uu, Lu, xx, uu, qq)

# Mx = jacobian(xx, M, xx)
# Mxx = jacobian(xx, Mx, xx)

#28 (generic function with 1 method)

In [5]:
T = 10
tspan = (0.0,T)
tt = 0:0.001:T

# mu = LinearInterpolation(zeros(m, length(tt)), tt)
# mu = LinearInterpolation(hcat(map(t->[0.05*sin(t); 0; 0.05*sin(t); 0], tt)...), tt)
# u_guess = zeros(4,length(tt))
# u_guess[1,401:end] = 5*blackman(601)
# u_guess[3,1:601] = 5*blackman(601)
mu = LinearInterpolation(hcat(map(t->0.05*sin(t), tt)...), tt)


dal(al,_,t) = f(al,mu(t))
al0 = x0
al = solve(ODEProblem(dal,al0,tspan))

Pr_terminal(al) = I(2*n) - inprod(al)
Ar(t) = fx(al(t),mu(t))
Br(t) = fu(al(t),mu(t))
Qr(t) = I(2*n) - inprod(al(t))
Rr(t) = 1

Kr(Pr,Rr,Br)= Rr\(Br'*Pr)
Kr(t) = Kr(Pr(t),Rr(t),Br(t))

dPr(Pr,_,t) = Ar(T-t)'*Pr + Pr*Ar(T-t) - Kr(Pr,Rr(T-t),Br(T-t))'*Rr(T-t)*Kr(Pr,Rr(T-t),Br(T-t)) + Qr(T-t)
Pr(t) = Pr_back(T-t)

Pr (generic function with 1 method)

In [6]:
u(x,Kr,al,mu) = mu - (Kr*(x - al))
u(t) = u(x(t),Kr(t),al(t),mu(t))
    
dx(x,_,t) = f(x,u(x,Kr(Pr(t),Rr(t),Br(t)),al(t),mu(t)))

dJ(_,_,t) = 0.01/2*u(t)'*u(t) + Qm*1/2*x(t)'*Qe*x(t)

P_terminal = Pf

A(t) = fx(x(t),u(t))
B(t) = fu(x(t),u(t))
a(t) = lx(x(t),u(t))
b(t) = lu(x(t),u(t))
Q(t) = lxx(x(t),u(t))
S(t) = lxu(x(t),u(t))
R(t) = luu(x(t),u(t))

dq(q,_,t) = (A(T-t) - B(T-t)*Kr(T-t))'*q + a(T-t) - Kr(T-t)'*b(T-t)
q(t) = q_back(T-t)
# qT(al) = Mx(al)' 


Qo(t) = Lxx(x(t),u(t),q(t))
So(t) = Lxu(x(t),u(t),q(t))
Ro(t) = Luu(x(t),u(t),q(t))

Ro (generic function with 1 method)

In [7]:
# GD
Ko(P,R,S,B)= R\(S' + B'*P)

Ko(t) = Ko(P(t),R(t),S(t),B(t))

dP(P,_,t) = A(T-t)'*P + P*A(T-t) - Ko(P,R(T-t),S(T-t),B(T-t))'*R(T-t)*Ko(P,R(T-t),S(T-t),B(T-t)) + Q(T-t)
P(t)=P_back(T-t)

PT = Pf

dr(r,_,t) = (A(T-t) - B(T-t)*Ko(T-t))'*r + a(T-t) - Ko(T-t)'*b(T-t)
r(t)=r_back(T-t)

vo(R,B,r,b) = -R\(B'*r + b)

vo(t) = vo(R(t),B(t),r(t),b(t))

v(z,Ko,vo) = (-Ko*z) + vec(vo)

v(t) = v(z(t),Ko(t),vo(t))

dz(z,_,t) = A(t)*z + B(t)*v(z,Ko(t),vo(t))

z0 = 0*x0

dy1(_,_,t) = a(t)'*z(t) + b(t)'*v(t)

dy2(_,_,t) = z(t)'*Qo(t)*z(t) + (2*z(t)'*So(t)*v(t))'[1] + v(t)'*Ro(t)*v(t)

gamma = 1

al1(x,z)    = x + gamma*z
mu1(u,v)    = u + gamma*v
u1(x1,Kr,al1,mu1) = mu1 - (Kr*(x1 - al1))

al1(t) = x(t) + gamma*z(t)
mu1(t) = mu1(u(t),v(t))
u1(t) = u1(x1(t),Kr(t),al1(t),mu1(t));


dx1(x1,_,t) = f(x1,u1(x1,Kr(t),al1(x(t),z(t)),mu1(u(t),v(t))))
dJ1(_,_,t) = 0.01/2*u1(t)'*u1(t) + Qm*1/2*x1(t)'*Qe*x1(t)

dJ1 (generic function with 1 method)

In [8]:
# Newt
Ko2(P2,Ro,So,B)= Ro\(So' + B'*P2)

Ko2(t) = Ko2(P2(t),Ro(t),So(t),B(t))

dP2(P2,_,t) = A(T-t)'*P2 + P2*A(T-t) - Ko2(P2,Ro(T-t),So(T-t),B(T-t))'*Ro(T-t)*Ko2(P2,Ro(T-t),So(T-t),B(T-t)) + Qo(T-t)
P2(t)=P2_back(T-t)

P2T = Pf

dr2(r2,_,t) = (A(T-t) - B(T-t)*Ko2(T-t))'*r2 + a(T-t) - Ko2(T-t)'*b(T-t)
r2(t)=r2_back(T-t)

vo2(Ro,B,r2,b) = -Ro\(B'*r2 + b)

vo2(t) = vo2(Ro(t),B(t),r2(t),b(t))

v2(z2,Ko2,vo2) = (-Ko2*z2) + vec(vo2)

v2(t) = v2(z2(t),Ko2(t),vo2(t))

dz2(z2,_,t) = A(t)*z2 + B(t)*v2(z2,Ko2(t),vo2(t))

z20 = 0*x0

dy12(_,_,t) = a(t)'*z2(t) + b(t)'*v2(t)

dy22(_,_,t) = z2(t)'*Qo(t)*z2(t) + (2*z2(t)'*So(t)*v2(t))'[1] + v2(t)'*Ro(t)*v2(t)

gamma = 1

al2(x,z2)    = x + gamma*z2
mu2(u,v2)    = u + gamma*v2
u2(x2,Kr,al2,mu2) = mu2 - Kr*(x2 - al2)

al2(t) = x(t) + gamma*z2(t)
mu2(t) = mu2(u(t),v2(t))
u2(t) = u2(x2(t),Kr(t),al2(t),mu2(t));


dx2(x2,_,t) = f(x2,u2(x2,Kr(t),al2(x(t),z2(t)),mu2(u(t),v2(t))))
dJ2(_,_,t) = 0.01/2*u2(t)'*u2(t) + Qm*1/2*x2(t)'*Qe*x2(t)

dJ2 (generic function with 1 method)

In [9]:
global Pr_back
global x
global J
global h
global q_back
global P_back
global P2_back
global r_back
global r2_back
global z
global z2
global y1
global y12
global y2
global y22
global Dh
global Dg2
global x1
global x2
global J1
global J2
global g
global tt
global utt
CostH = zeros(20)
CostG = zeros(20)
Descent = zeros(20)
Gam = zeros(20)
Method = zeros(20)

utt = zeros(m,length(tt))

for iter = 1:20

    Pr_back = solve(ODEProblem(dPr,Pr_terminal(al(T)),tspan))

    x = solve(ODEProblem(dx,x0,tspan))

    J = solve(ODEProblem(dJ,0,tspan))

    h = J(T) + 1/2*x(T)'*Pf*x(T)
    
    CostH[iter] = h

    q_back = solve(ODEProblem(dq,Pf*al(T),tspan))
        
    P2_back = solve(ODEProblem(dP2,P2T,tspan),maxiters = 1e3)
    
    if abs(P2_back.destats.maxeig) < 1e8
        
        r2_back = solve(ODEProblem(dr2,Pf*al(T),tspan))
        
        z2 = solve(ODEProblem(dz2,z20,tspan))
        
        y12 = solve(ODEProblem(dy12,0,tspan))
        
        y22 = solve(ODEProblem(dy22,0,tspan))
        
        Dh = y12(T) + (Pf*al(T))'*z2(T)
        Dg2 = y22(T) + z2(T)'*Pf*z2(T)
    
        Descent[iter] = -Dh
        
        if Descent[iter] < 1e-5
            break
        end
        
        Method[iter] = 2

        x2 = al
        J2 = J
        g = h
        gamma = 1

        for j = 1:10

            x2 = solve(ODEProblem(dx2,x0,tspan))

            J2 = solve(ODEProblem(dJ2,0,tspan))

            g = J2(T) + 1/2*x2(T)'*Pf*x2(T)
    
            if h-g >= -0.4*gamma*Dh
                Gam[iter] = gamma
                break
            else
                gamma = 0.7*gamma
            end
    
        end
    
        CostG[iter] = g

    
        for counter = 1:length(tt)
            utt[:,counter] = u2(tt[counter])
        end
    
        mu = LinearInterpolation(utt,tt)
        al = x2
        
    else 
        
        P_back = solve(ODEProblem(dP,PT,tspan))

        r_back = solve(ODEProblem(dr,Pf*al(T),tspan))

        z = solve(ODEProblem(dz,z0,tspan))

        y1 = solve(ODEProblem(dy1,0,tspan))

        y2 = solve(ODEProblem(dy2,0,tspan))

        Dh = y1(T) + (Pf*al(T))'*z(T)
        Dg2 = y2(T) + z(T)'*Pf*z(T)
    
        Descent[iter] = -Dh
        
        if Descent[iter] < 1e-5
            break
        end
        
        Method[iter] = 1

        x1 = al
        J1 = J
        g = h
        gamma = 1

        for j = 1:10

            x1 = solve(ODEProblem(dx1,x0,tspan))

            J1 = solve(ODEProblem(dJ1,0,tspan))

            g = J1(T) + 1/2*x1(T)'*Pf*x1(T)
    
            if h-g >= -0.4*gamma*Dh
                Gam[iter] = gamma
                break
            else
                gamma = 0.7*gamma
            end
    
        end
    
        CostG[iter] = g

    
        for counter = 1:length(tt)
            utt[:,counter] = u1(tt[counter])
        end
    
        mu = LinearInterpolation(utt,tt)
        al = x1
        
    end
    
end

┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /Users/jieqiushao/.julia/packages/SciMLBase/h4Gxc/src/integrator_interface.jl:345
┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /Users/jieqiushao/.julia/packages/SciMLBase/h4Gxc/src/integrator_interface.jl:345
┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /Users/jieqiushao/.julia/packages/SciMLBase/h4Gxc/src/integrator_interface.jl:345
┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /Users/jieqiushao/.julia/packages/SciMLBase/h4Gxc/src/integrator_interface.jl:345
┌ Warning: dt <= dtmin. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBa

In [10]:
Descent

20-element Vector{Float64}:
 0.43131845438085625
 0.25692936437001246
 0.16860266966749826
 0.13513362617394808
 0.09857836652608429
 0.07253174337964159
 0.04456130685778886
 0.02122118208795478
 0.01797074173521849
 0.020165348212987613
 0.02179272534107675
 0.02221880960251818
 0.02169664338505584
 0.01747377345434242
 0.009028659129185612
 6.734977017275448e-5
 1.2597223543153798e-6
 0.0
 0.0
 0.0

In [11]:
Method

20-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0

In [12]:
Gam

20-element Vector{Float64}:
 0.24009999999999992
 0.24009999999999992
 0.3429999999999999
 0.16806999999999994
 0.11764899999999995
 0.3429999999999999
 0.7
 0.48999999999999994
 0.3429999999999999
 0.16806999999999994
 0.11764899999999995
 0.16806999999999994
 0.24009999999999992
 0.48999999999999994
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0

In [13]:
using MAT

file = matopen("Uopt.mat", "w")
write(file, "Uopt", utt)
close(file)